In [74]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [75]:
df = pd.read_csv("C:\\Users\\ajaco\\Desktop\\repos\\noreallyimfine\\ebird-project\\labelled_bird_sample.csv")

print(df.shape)

df.head()

(103992, 15)


,name,observ_count,country,state,county,latitude,longitude,observ_date,month,season,county_state,State,CountyName,RegionName,seas_reg_rare
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,8,Summer,CookIllinois,Illinois,Cook,Great Lakes,Uncommon
1,Great Blue Heron,2,United States,Illinois,Cook,41.956746,-87.913027,1999-07-24,7,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
2,Purple Martin,6,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
3,Common Yellowthroat,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common
4,Eastern Towhee,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common


In [76]:
df['target'] = df['seas_reg_rare'].map({"Common": 0,
                                        "Uncommon": 1,
                                        "Rare": 2})

df.head()

,name,observ_count,country,state,county,latitude,longitude,observ_date,month,season,county_state,State,CountyName,RegionName,seas_reg_rare,target
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,8,Summer,CookIllinois,Illinois,Cook,Great Lakes,Uncommon,1
1,Great Blue Heron,2,United States,Illinois,Cook,41.956746,-87.913027,1999-07-24,7,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0
2,Purple Martin,6,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0
3,Common Yellowthroat,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0
4,Eastern Towhee,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0


In [77]:
df.target.value_counts(normalize=True)

0    0.654406
1    0.304495
2    0.041099
Name: target, dtype: float64

In [78]:
import category_encoders as ce

In [79]:
train, test = train_test_split(df, test_size=0.2, stratify=df['seas_reg_rare'])

train.shape, test.shape

((83193, 16), (20799, 16))

In [80]:
features = ['name', 'season', 'RegionName', 'latitude', 'longitude']
target = 'target'

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((83193, 5), (83193,), (20799, 5), (20799,))

## CatBoost Encoder

In [97]:
df['target'] = df['seas_reg_rare'].map({"Common": 2,
                                        "Uncommon": 1,
                                        "Rare": 0})

In [119]:
features = ['name', 'season', 'RegionName', 'latitude', 'longitude']
target = 'target'

X = df[features]
y = df[target]

encoder = ce.CatBoostEncoder()

X = encoder.fit_transform(X, y)

X

,name,season,RegionName,latitude,longitude
0,1.613307,1.613307,1.613307,41.775629,-87.583273
1,1.613307,1.306653,1.306653,41.956746,-87.913027
2,1.613307,1.537769,1.537769,41.707907,-87.886848
3,1.613307,1.653327,1.653327,41.707907,-87.886848
4,1.613307,1.722661,1.722661,41.707907,-87.886848
...,...,...,...,...,...
103987,1.337333,1.617486,1.666266,34.426782,-97.149598
103988,1.832528,1.574056,1.666069,34.426782,-97.149598
103989,1.339844,1.574067,1.628092,31.667925,-81.845470
103990,1.945871,1.574052,1.666108,35.116398,-87.208443


In [120]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((83193, 5), (20799, 5), (83193,), (20799,))

### Random Forest 

    (1000 trees, 'entropy' criterion, No max_depth)

In [123]:
forest = RandomForestClassifier(n_estimators=1000, max_features=0.25, max_depth=None, random_state=42)

forest.fit(X_train, y_train)

forest.score(X_test, y_test)

0.7907110918794172

In [104]:
from sklearn.metrics import roc_auc_score, confusion_matrix, recall_score, precision_score, accuracy_score

In [124]:
preds = forest.predict(X_test)

print("Random Forest Accuracy Score", accuracy_score(y_test, preds))
print("Random Forest Recall Score", recall_score(y_test, preds, average='weighted'))
print("Random Forest Precision Score", precision_score(y_test, preds, average='weighted'))
#print("Random Forest ROC-AUC Score", roc_auc_score(y_test, preds, average='weighted', multi_class='ovo'))

Random Forest Accuracy Score 0.7907110918794172
Random Forest Recall Score 0.7907110918794172
Random Forest Precision Score 0.7768295401468783


### Random Forest w/o Lat/Lon

In [111]:
features = ['name', 'season', 'RegionName']
target = 'target'

X = df[features]
y = df[target]

encoder = ce.CatBoostEncoder()

X = encoder.fit_transform(X, y)

X

,name,season,RegionName
0,1.613307,1.613307,1.613307
1,1.613307,1.306653,1.306653
2,1.613307,1.537769,1.537769
3,1.613307,1.653327,1.653327
4,1.613307,1.722661,1.722661
...,...,...,...
103987,1.337333,1.617486,1.666266
103988,1.832528,1.574056,1.666069
103989,1.339844,1.574067,1.628092
103990,1.945871,1.574052,1.666108


In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((83193, 3), (20799, 3), (83193,), (20799,))

In [113]:
forest = RandomForestClassifier(n_estimators=1000, max_depth=None, random_state=42)

forest.fit(X_train, y_train)

forest.score(X_test, y_test)

0.7792201548151353

In [114]:
preds = forest.predict(X_test)

print("Random Forest Accuracy Score", accuracy_score(y_test, preds))
print("Random Forest Recall Score", recall_score(y_test, preds, average='weighted'))
print("Random Forest Precision Score", precision_score(y_test, preds, average='weighted'))
#print("Random Forest ROC-AUC Score", roc_auc_score(y_test, preds, average='weighted', multi_class='ovo'))

Random Forest Accuracy Score 0.7792201548151353
Random Forest Recall Score 0.7792201548151353
Random Forest Precision Score 0.7653383477960596


### RF including CountyState

In [115]:
features = ['name', 'season', 'RegionName', 'latitude', 'longitude', 'county_state']
target = 'target'

X = df[features]
y = df[target]

encoder = ce.CatBoostEncoder()

X = encoder.fit_transform(X, y)

X

,name,season,RegionName,latitude,longitude,county_state
0,1.613307,1.613307,1.613307,41.775629,-87.583273,1.613307
1,1.613307,1.306653,1.306653,41.956746,-87.913027,1.306653
2,1.613307,1.537769,1.537769,41.707907,-87.886848,1.537769
3,1.613307,1.653327,1.653327,41.707907,-87.886848,1.653327
4,1.613307,1.722661,1.722661,41.707907,-87.886848,1.722661
...,...,...,...,...,...,...
103987,1.337333,1.617486,1.666266,34.426782,-97.149598,1.613307
103988,1.832528,1.574056,1.666069,34.426782,-97.149598,0.806653
103989,1.339844,1.574067,1.628092,31.667925,-81.845470,1.613307
103990,1.945871,1.574052,1.666108,35.116398,-87.208443,1.613307


In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((83193, 6), (20799, 6), (83193,), (20799,))

In [117]:
forest = RandomForestClassifier(n_estimators=1000, max_depth=None, random_state=42)

forest.fit(X_train, y_train)

forest.score(X_test, y_test)

0.7913361219289389

In [118]:
preds = forest.predict(X_test)

print("Random Forest Accuracy Score", accuracy_score(y_test, preds))
print("Random Forest Recall Score", recall_score(y_test, preds, average='weighted'))
print("Random Forest Precision Score", precision_score(y_test, preds, average='weighted'))
#print("Random Forest ROC-AUC Score", roc_auc_score(y_test, preds, average='weighted', multi_class='ovo'))

Random Forest Accuracy Score 0.7913361219289389
Random Forest Recall Score 0.7913361219289389
Random Forest Precision Score 0.7776867687549218


In [84]:
from sklearn.model_selection import GridSearchCV

In [85]:
param_grid = {
    "n_estimators": [100, 250, 500, 1000],
    "max_depth": [5,10,15],
    "max_features": [.25, .5, .75, 1.0]
}

forest = RandomForestClassifier(random_state=42)
grid = GridSearchCV(forest, param_grid, n_jobs=-1, cv=3, verbose=10)

grid.fit(X, y)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   48.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 16.9min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 20.5min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 26.5min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 36.1min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 47.4min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 54.8min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 68.2min
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed: 82.6min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=42,
                                  

In [88]:
grid.best_score_

0.7731940918532195

In [89]:
grid.best_params_

{'max_depth': 10, 'max_features': 0.75, 'n_estimators': 1000}

In [96]:
forest = RandomForestClassifier(n_estimators=1000, max_depth=None, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

forest.fit(X_train, y_train)

forest.score(X_test, y_test)

0.7957113322755902

### Gradient Boosting

In [91]:
from sklearn.ensemble import GradientBoostingClassifier

In [93]:
# boost = GradientBoostingClassifier(random_state=42)

# param_grid = {
#     "n_estimators": [100, 500, 1000],
#     "learning_rate": [0.001, 0.01, 0.1],
#     "max_depth": [3, 5, 10, None]
# }

# grid = GridSearchCV(boost, param_grid, n_jobs=-1, cv=3, verbose=10)

# grid.fit(X, y)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 17.0min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 45.1min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 75.4min


KeyboardInterrupt: 

In [94]:
boost = GradientBoostingClassifier(n_estimators=500, learning_rate=0.01, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

boost.fit(X_train, y_train)

boost.score(X_test, y_test)

0.7713832395788259

## Hashing Encoder

In [62]:
features = ['name', 'season', 'RegionName', 'latitude', 'longitude']
target = 'target'

X = df[features]
y = df[target]

encoder = ce.HashingEncoder()

X = encoder.fit_transform(X)

X

,name,season,RegionName,latitude,longitude
0,Magnolia Warbler,Summer,Great Lakes,41.775629,-87.583273
1,Great Blue Heron,Summer,Great Lakes,41.956746,-87.913027
2,Purple Martin,Summer,Great Lakes,41.707907,-87.886848
3,Common Yellowthroat,Summer,Great Lakes,41.707907,-87.886848
4,Eastern Towhee,Summer,Great Lakes,41.707907,-87.886848
...,...,...,...,...,...
103987,Inca Dove,Summer,Appohzarka,34.426782,-97.149598
103988,Eastern Phoebe,Spring,Appohzarka,34.426782,-97.149598
103989,Wood Stork,Spring,Deep South,31.667925,-81.845470
103990,Eastern Towhee,Spring,Appohzarka,35.116398,-87.208443


In [65]:
encoder.fit(X).transform(X)

,name,season,RegionName,latitude,longitude
0,Magnolia Warbler,Summer,Great Lakes,41.775629,-87.583273
1,Great Blue Heron,Summer,Great Lakes,41.956746,-87.913027
2,Purple Martin,Summer,Great Lakes,41.707907,-87.886848
3,Common Yellowthroat,Summer,Great Lakes,41.707907,-87.886848
4,Eastern Towhee,Summer,Great Lakes,41.707907,-87.886848
...,...,...,...,...,...
103987,Inca Dove,Summer,Appohzarka,34.426782,-97.149598
103988,Eastern Phoebe,Spring,Appohzarka,34.426782,-97.149598
103989,Wood Stork,Spring,Deep South,31.667925,-81.845470
103990,Eastern Towhee,Spring,Appohzarka,35.116398,-87.208443


In [66]:
encoder

HashingEncoder(cols=['name', 'season', 'RegionName'], drop_invariant=False,
               hash_method='md5', max_process=2, max_sample=51996,
               n_components=8, return_df=True, verbose=0)

## Target Encoding

In [34]:
encoder = ce.TargetEncoder()

features = ['name', 'season', 'RegionName', 'latitude', 'longitude']
target = 'target'

X = df[features]
y = df[target]


X = encoder.fit_transform(X, y)

In [35]:
X

,name,season,RegionName,latitude,longitude
0,0.717277,0.382578,0.351550,41.775629,-87.583273
1,0.007663,0.382578,0.351550,41.956746,-87.913027
2,0.502128,0.382578,0.351550,41.707907,-87.886848
3,0.105183,0.382578,0.351550,41.707907,-87.886848
4,0.053305,0.382578,0.351550,41.707907,-87.886848
...,...,...,...,...,...
103987,0.685714,0.382578,0.333846,34.426782,-97.149598
103988,0.166667,0.425927,0.333846,34.426782,-97.149598
103989,0.674419,0.425927,0.371935,31.667925,-81.845470
103990,0.053305,0.425927,0.333846,35.116398,-87.208443


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((83193, 5), (20799, 5), (83193,), (20799,))

In [40]:
forest = RandomForestClassifier()

forest.fit(X_train, y_train)

forest.score(X_test, y_test)

0.9101399105726237

In [44]:
preds = forest.predict(X_test)

pred_df = pd.DataFrame({"bird": X_test['name'],
              "season": X_test['season'],
              "region": X_test['RegionName'],
              "true_label": y_test,
              "pred_label": preds})

In [46]:
pred_df.pred_label.value_counts()

0    14334
1     6102
2      363
Name: pred_label, dtype: int64

In [47]:
y_test.value_counts()

0    13723
1     6186
2      890
Name: target, dtype: int64

In [48]:
dir(encoder)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_dim',
 '_get_param_names',
 '_get_tags',
 '_mean',
 '_more_tags',
 'cols',
 'drop_cols',
 'drop_invariant',
 'feature_names',
 'fit',
 'fit_target_encoding',
 'fit_transform',
 'get_feature_names',
 'get_params',
 'handle_missing',
 'handle_unknown',
 'mapping',
 'min_samples_leaf',
 'ordinal_encoder',
 'return_df',
 'set_params',
 'smoothing',
 'target_encode',
 'transform',
 'verbose']

In [61]:
encoder.fit_target_encoding()

TypeError: fit_target_encoding() missing 2 required positional arguments: 'X' and 'y'

In [55]:
encoder.mapping

{'name': name
  1      0.717277
  2      0.007663
  3      0.502128
  4      0.105183
  5      0.053305
           ...   
  788    0.386693
  789    0.386693
  790    0.386693
 -1      0.386693
 -2      0.386693
 Length: 792, dtype: float64,
 'season': season
  1    0.382578
  2    0.425927
  3    0.400542
  4    0.284406
 -1    0.386693
 -2    0.386693
 dtype: float64,
 'RegionName': RegionName
  1     0.351550
  2     0.404454
  3     0.525745
  4     0.515620
  5     0.314624
  6     0.231455
  7     0.371935
  8     0.333846
  9     0.336313
  10    0.364355
  11    0.394645
  12    0.466930
  13    0.364209
  14    0.337215
 -1     0.386693
 -2     0.386693
 dtype: float64}

In [6]:
from sklearn.cluster import KMeans

In [12]:
encoder = ce.OrdinalEncoder()

cat_data = encoder.fit_transform(df[['name', 'season', 'RegionName']])
num_data = df[['latitude', 'longitude']].values

# data = cat_data.values + num_data
# data

ValueError: operands could not be broadcast together with shapes (103992,3) (103992,2) 

In [14]:
kmeans = KMeans(n_clusters=3)

groups = kmeans.fit_predict(cat_data)

In [15]:
df['pred_groups'] = groups

df.head()

,name,observ_count,country,state,county,latitude,longitude,observ_date,month,season,county_state,State,CountyName,RegionName,seas_reg_rare,pred_groups
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,8,Summer,CookIllinois,Illinois,Cook,Great Lakes,Uncommon,0
1,Great Blue Heron,2,United States,Illinois,Cook,41.956746,-87.913027,1999-07-24,7,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0
2,Purple Martin,6,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0
3,Common Yellowthroat,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0
4,Eastern Towhee,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,0


In [17]:
df.pred_groups.value_counts(normalize=True)

0    0.539378
2    0.315149
1    0.145473
Name: pred_groups, dtype: float64

In [18]:
from sklearn.metrics import confusion_matrix

In [20]:
confusion_matrix(df['seas_reg_rare'].map({"Common": 0,
                                          "Uncommon": 1,
                                          "Rare": 2}), df['pred_groups'])



array([[43422,  5594, 19037],
       [11674,  7587, 12404],
       [  995,  1947,  1332]], dtype=int64)

In [21]:
from sklearn.preprocessing import OneHotEncoder

In [22]:
encoder = OneHotEncoder()

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

ValueError: Found unknown categories ['Antillean Nighthawk', 'Kalij Pheasant', 'Great Skua', 'Corn Crake', 'Little Curlew', 'Kauai Elepaio', 'Masked Duck', 'Passenger Pigeon', "Bulwer's Petrel"] in column 0 during transform

### Hashing Encoder

- breaking away from this cuz its not working

In [7]:
encoder = ce.HashingEncoder()

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [8]:
X_train

,name,season,RegionName,latitude,longitude
68458,Red-necked Phalarope,Fall,Heartland,45.177800,-93.501600
51708,House Finch,Spring,Mountain West,36.030221,-112.851305
43560,House Finch,Spring,Heartland,43.045578,-89.466100
22842,Mourning Dove,Winter,West Coast,32.759226,-117.221841
59836,Common Raven,Summer,Northwest,39.227499,-123.764377
...,...,...,...,...,...
103260,Northern Rough-winged Swallow,Summer,Appohzarka,34.724472,-92.761774
62751,Turkey Vulture,Spring,Southern Florida,26.949311,-81.030693
3344,Mallard,Spring,Mid-Atlantic North,39.397850,-76.986394
86682,Red-eyed Vireo,Spring,Northeast,42.368437,-74.439239


In [14]:
forest = RandomForestClassifier()

forest.fit(X_train, y_train)
forest.score(X_test, y_test)

ValueError: could not convert string to float: 'Northern Harrier'